In [8]:
import json
import os

with open('benchmark_results_COARENING_POST_PROCESSED_limit8_add.json', 'r') as file:
    data = json.load(file)


sorted_results = {}

for element in data:

    num_qubits = element['num_qubits']
    network_type = element['network_type']
    num_partitions = element['num_partitions']
    coarsening_factor = element['coarsening_factor']
    level_limit = element['level_limit']
    initial_cost_no_grouping = element['initial_cost_no_grouping']
    initial_cost = element['initial_cost']
    final_cost = element['final_cost']
    iteration = element['iteration']
    time_taken = element['time_taken']
    seed = element['seed']


    key = (num_qubits, network_type, num_partitions, coarsening_factor)
    
    if key not in sorted_results:
        sorted_results[key] = []

    sorted_results[key].append({
        'initial_cost_no_grouping': initial_cost_no_grouping,
        'initial_cost': initial_cost,
        'final_cost': final_cost,
        'iteration': iteration,
        'time_taken': time_taken
    })


In [9]:
mean_results_sorted = {}

for key in sorted_results:

    num_qubits, network_type, num_partitions, coarsening_factor = key
    entry = sorted_results[key]
    valid_entries = [e for e in entry if e['final_cost'] is not None]
    if not valid_entries:
        continue  # Skip if there are no valid entries
    mean_cost_init_ng = sum(e['initial_cost_no_grouping'] for e in valid_entries) / len(valid_entries)
    mean_cost_init = sum(e['initial_cost'] for e in valid_entries) / len(valid_entries)
    mean_cost_final = sum(e['final_cost'] for e in valid_entries) / len(valid_entries)
    mean_time_taken = sum(e['time_taken'] for e in valid_entries) / len(valid_entries)

    mean_results_sorted[key] = {
        'num_partitions': num_partitions,
        'coarsening_factor': coarsening_factor,
        'mean_cost_init_ng': mean_cost_init_ng,
        'mean_cost_init': mean_cost_init,
        'mean_cost_final': mean_cost_final,
        'mean_time_taken': mean_time_taken
    }


# Now to write the results to separate .dat files based on the number of qubits and network type

for key, value in mean_results_sorted.items():
    num_qubits, network_type, num_partitions, coarsening_factor = key
    filename = f"mean_results_{num_qubits}_{network_type}.dat"
    if not os.path.exists(filename):
        with open(filename, 'w') as file:
            file.write("num_partitions coarsening_factor mean_cost_init_ng mean_cost_init mean_cost_final mean_time_taken\n")
    with open(filename, 'a') as file:
        file.write(f"{num_partitions} {coarsening_factor} {value['mean_cost_init_ng']} {value['mean_cost_init']} {value['mean_cost_final']} {value['mean_time_taken']}\n")
